<a href="https://colab.research.google.com/github/ydev365/portfolio-for-internship/blob/main/firecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


이 코드는 그냥 8명 단체단톡방인 FIRECAST에 올려둔 call119_train1 로 돌려봤는데 성능이 RSME 가 1.6 이다. RSME 라는 숫자가 낮을수록 성능이 좋은거다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/call119_train1.csv'
df = pd.read_csv(file_path)

# --- Start of modifications for handling missing values using ffill ---

# List of columns identified to have -99.0 as a placeholder for missing values
# Based on the previous analysis of the CSV file.
columns_to_impute = [
    'call119_train.ta_max',
    'call119_train.ta_min',
    'call119_train.ta_max_min',
    'call119_train.hm_min',
    'call119_train.hm_max',
    'call119_train.ws_max',
    'call119_train.ws_ins_max',
    'call119_train.rn_day'
]

# Replace -99.0 with NaN in the identified columns
for col in columns_to_impute:
    df[col] = df[col].replace(-99.0, np.nan)

# --- End of modifications for handling missing values ---

# Separate target variable Y from features X
X = df.drop('call119_train.call_count', axis=1)
y = df['call119_train.call_count']

# Drop 'call119_train.address_city' column as specified
X = X.drop('call119_train.address_city', axis=1)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to 'call119_train.address_gu' and 'call119_train.sub_address'
# These are handled before imputation because they are categorical and ffill is for numerical.
X['call119_train.address_gu'] = label_encoder.fit_transform(X['call119_train.address_gu'])
X['call119_train.sub_address'] = label_encoder.fit_transform(X['call119_train.sub_address'])

# --- New step: Impute missing values with forward fill (ffill) ---
# Apply ffill to all numerical columns that might contain NaNs after placeholder replacement.
# Note: If there are NaNs at the very beginning of a column, ffill will not fill them.
# In such cases, a bfill (backward fill) or mean/median imputation might be needed as a fallback.
X = X.fillna(method='ffill')

# Optional: If there are still NaNs after ffill (e.g., if the first value was NaN),
# you might want to fill remaining NaNs with the mean or 0 as a last resort.
# X = X.fillna(X.mean(numeric_only=True))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the RandomForestRegressor model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f'검증 데이터의 RMSE: {rmse}')

<ipython-input-9-2129252055>:52: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X = X.fillna(method='ffill')


검증 데이터의 RMSE: 1.6767286110690767


이거는 이제 제가 전처리한 파일인 preprocessed_full_data_V3.csv 저희 4명 팀즈 채팅방에 올려놓은 걸로 돌렸더니 RSME 가 1.48로 개선됐다. 전처리를 잘해야 예측값이 더 정확해지는구나. 하지만 나중가서 안사실은 결측치를 비워두고 모델이 알아서 처리하게끔해야 성능이 더 잘나오는 것도 있다. 사람이 결측치를 처리하는것보다 기계가 결측치를 처리할때 성능이 더 잘뽑힐때가 많다.

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Load the preprocessed dataset
# Load the dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)

# Extract year from 'call119_train.tm'
# Assuming 'call119_train.tm' is an integer like 2020010100
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])]
test_df = df[df['year'] == 2023]

# Define features (X) and target (Y)
# Exclude 'Unnamed: 0', 'year', and the target 'call119_train.call_count' from features
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features]
X_train=X_train.drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features]
X_test=X_test.drop('call119_train.tm', axis=1)
Y_test = test_df[target]







# Initialize and train RandomForestRegressor model
# Set random_state for reproducibility
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = rf_model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"RMSE: {rmse}")

#RMSE: 1.4809474772084894

RMSE: 1.4809474772084894


이 코드는 제가 월,일 날짜 데이터를 넣은채 돌렸더니 성능이 약간 안좋아졌다. 그냥 날짜데이터를 아예 뺀게 애초에 더 좋은 성능을 나타냈다.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Load the dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)


# Extract year from 'call119_train.tm' for splitting
# Assuming 'call119_train.tm' is an integer like 2020010100
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy() # .copy() to avoid SettingWithCopyWarning
test_df = df[df['year'] == 2023].copy() # .copy() to avoid SettingWithCopyWarning

# Define features (X) and target (Y)
# Exclude 'Unnamed: 0', 'year', and the target 'call119_train.call_count' from features
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].copy()
Y_train = train_df[target].copy()
X_test = test_df[features].copy()
Y_test = test_df[target].copy()

# Modify 'call119_train.tm' in X_train and X_test
# Convert to string, remove the '.0', then take the last 4 digits (MMDD)
X_train['call119_train.tm'] = X_train['call119_train.tm'].astype(str).str.replace('.0', '', regex=False).str[-4:].astype(int)
X_test['call119_train.tm'] = X_test['call119_train.tm'].astype(str).str.replace('.0', '', regex=False).str[-4:].astype(int)

# Initialize and train RandomForestRegressor model
# Set random_state for reproducibility
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, Y_train)

# Make predictions on the test set
Y_pred = rf_model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"RMSE after modifying 'call119_train.tm': {rmse}")

RMSE after modifying 'call119_train.tm': 1.49491462326673


XGboost 하이퍼파라미터 조정하니까 이것도 RSME 1.26으로 괜찮게 나왔다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
# XGBoost 임포트
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)

# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features].drop('call119_train.tm', axis=1)
Y_test = test_df[target]

# --- RandomForestRegressor 결과 ---
# rf_model = RandomForestRegressor(random_state=42)
# rf_model.fit(X_train, Y_train)
# Y_pred_rf = rf_model.predict(X_test)
# rmse_rf = np.sqrt(mean_squared_error(Y_test, Y_pred_rf))
# print(f"RandomForestRegressor RMSE: {rmse_rf}")

# --- LightGBM 결과 ---
# lgb_model = lgb.LGBMRegressor(random_state=42)
# lgb_model.fit(X_train, Y_train)
# Y_pred_lgb = lgb_model.predict(X_test)
# rmse_lgb = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb))
# print(f"LGBMRegressor RMSE: {rmse_lgb}")


# --- XGBoost 모델 ---
# print("\n--- XGBoost (XGBRegressor) ---")
# # objective='reg:squarederror'는 회귀 문제에서 제곱 오차를 최소화하도록 지정
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
# xgb_model.fit(X_train, Y_train)

# Y_pred_xgb = xgb_model.predict(X_test)
# rmse_xgb = np.sqrt(mean_squared_error(Y_test, Y_pred_xgb))
# print(f"XGBRegressor RMSE: {rmse_xgb}")

# XGBoost 하이퍼파라미터 튜닝 예시 (간략히)
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7]
}
grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_xgb.fit(X_train, Y_train)
print(f"Best XGB parameters: {grid_search_xgb.best_params_}")
Y_pred_xgb_tuned = grid_search_xgb.best_estimator_.predict(X_test)
rmse_xgb_tuned = np.sqrt(mean_squared_error(Y_test, Y_pred_xgb_tuned))
print(f"XGBRegressor Tuned RMSE: {rmse_xgb_tuned}")

Best XGB parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 300}
XGBRegressor Tuned RMSE: 1.2617998126653973


이 밑의 코드는 하이퍼파라미터를 조정하니까 성능이 살짝 구리게 나왔다. RSME 가 1.24로 나왔다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
# LightGBM 임포트
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)

# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features].drop('call119_train.tm', axis=1)
Y_test = test_df[target]

# --- RandomForestRegressor 결과 (이전 코드) ---
# rf_model = RandomForestRegressor(random_state=42)
# rf_model.fit(X_train, Y_train)
# Y_pred_rf = rf_model.predict(X_test)
# rmse_rf = np.sqrt(mean_squared_error(Y_test, Y_pred_rf))
# print(f"RandomForestRegressor RMSE: {rmse_rf}")


# --- LightGBM 모델 ---
# print("\n--- LightGBM (LGBMRegressor) ---")
# lgb_model = lgb.LGBMRegressor(random_state=42)
# lgb_model.fit(X_train, Y_train)

# Y_pred_lgb = lgb_model.predict(X_test)
# rmse_lgb = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb))
# print(f"LGBMRegressor RMSE: {rmse_lgb}")

# LightGBM 하이퍼파라미터 튜닝 예시 (간략히)
param_grid_lgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [5, 10, 15]
}
grid_search_lgb = GridSearchCV(lgb_model, param_grid_lgb, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search_lgb.fit(X_train, Y_train)
print(f"Best LGBM parameters: {grid_search_lgb.best_params_}")
Y_pred_lgb_tuned = grid_search_lgb.best_estimator_.predict(X_test)
rmse_lgb_tuned = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb_tuned))
print(f"LGBMRegressor Tuned RMSE: {rmse_lgb_tuned}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001015 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 32372, number of used features: 10
[LightGBM] [Info] Start training from score 1.950358
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

 RSME 1.23로 나옴




In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
# XGBoost 임포트
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)

# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features].drop('call119_train.tm', axis=1)
Y_test = test_df[target]

# --- RandomForestRegressor 결과 ---
# rf_model = RandomForestRegressor(random_state=42)
# rf_model.fit(X_train, Y_train)
# Y_pred_rf = rf_model.predict(X_test)
# rmse_rf = np.sqrt(mean_squared_error(Y_test, Y_pred_rf))
# print(f"RandomForestRegressor RMSE: {rmse_rf}")

# --- LightGBM 결과 ---
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train, Y_train)
Y_pred_lgb = lgb_model.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb))
print(f"LGBMRegressor RMSE: {rmse_lgb}")


# --- XGBoost 모델 ---
# print("\n--- XGBoost (XGBRegressor) ---")
# # objective='reg:squarederror'는 회귀 문제에서 제곱 오차를 최소화하도록 지정
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
# xgb_model.fit(X_train, Y_train)

# Y_pred_xgb = xgb_model.predict(X_test)
# rmse_xgb = np.sqrt(mean_squared_error(Y_test, Y_pred_xgb))
# print(f"XGBRegressor RMSE: {rmse_xgb}")

# XGBoost 하이퍼파라미터 튜닝 예시 (간략히)
# param_grid_xgb = {
#     'n_estimators': [100, 200, 300],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7]
# }
# grid_search_xgb = GridSearchCV(xgb_model, param_grid_xgb, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
# grid_search_xgb.fit(X_train, Y_train)
# print(f"Best XGB parameters: {grid_search_xgb.best_params_}")
# Y_pred_xgb_tuned = grid_search_xgb.best_estimator_.predict(X_test)
# rmse_xgb_tuned = np.sqrt(mean_squared_error(Y_test, Y_pred_xgb_tuned))
# print(f"XGBRegressor Tuned RMSE: {rmse_xgb_tuned}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001053 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 32372, number of used features: 10
[LightGBM] [Info] Start training from score 1.950358
LGBMRegressor RMSE: 1.2333911206517731


범주형 칼럼을 인코딩 할 필요없이 칼럼의 object type을 category로 바꾸니까 성능이 크게 향상됐다. RSME 1.23=> 1.16 으로


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
# XGBoost 임포트
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/call119_train1.csv'
df = pd.read_csv(file_path)
df.replace(-99, np.nan, inplace=True)



# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()


# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count','call119_train.address_city','call119_train.stn']]
target = 'call119_train.call_count'



X_train = train_df[features].drop('call119_train.tm', axis=1)


#odject=> category 로 바꿔야 알아서 범주형 칼럼을 인코딩해준다.
for col in X_train.select_dtypes(include='object').columns:
    X_train[col] = X_train[col].astype('category')

Y_train = train_df[target]

#odject=> category 로 바꿔야 알아서 범주형 칼럼을 인코딩해준다.
X_test = test_df[features].drop('call119_train.tm', axis=1)

for coll in X_test.select_dtypes(include='object').columns:
    X_test[coll] = X_test[coll].astype('category')

Y_test = test_df[target]


# # --- LightGBM 결과 ---
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train, Y_train)
Y_pred_lgb = lgb_model.predict(X_test)
rmse_lgb = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb))
print(f"LGBMRegressor RMSE: {rmse_lgb}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003344 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1845
[LightGBM] [Info] Number of data points in the train set: 32372, number of used features: 10
[LightGBM] [Info] Start training from score 1.950358
LGBMRegressor RMSE: 1.1664874808829957


날짜 데이터를 아예 빼지말고 연도만 추출해서 넣으니까 RSME가 1.16=>1.15로 성능이 좋아짐. 단일 모델 데이터로는 이것이 최고로 나왔다. 연도를 넣으니까 성능이 더 향상되는 이유는 내 갠적 생각에는 코로나 때문인것 같다. 코로나는 2019부터 시작해서 2022년까지 우릴 괴롭히고 구급대원을 불러 신고한 사람들이 많았으니 이에따른 신고 증가 그리고 연도가 올라갈수록 코로나가 잠잠해져 뚜렷한 신고 감소가 나타났으므로, 각종 온도, 습도, 풍속 등의 날씨 데이터만으로는 신고의 추이가 반영이 안된것이 연도 정보와 함께 넣으니까 신고의 수가 반영된것같다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import lightgbm as lgb
# XGBoost 임포트
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/call119_train1.csv'
df = pd.read_csv(file_path)
df.replace(-99, np.nan, inplace=True)

# --- 1. 날짜 데이터 변환 및 특성 공학 ---
print("날짜 특성을 생성합니다...")
# 'call119_train.tm'을 datetime 객체로 변환합니다. (YYYYMMDD 형식이라고 가정)
df['call119_train.tm'] = pd.to_datetime(df['call119_train.tm'], format='%Y%m%d')

# 기본 날짜 특성 추출
df['year'] = df['call119_train.tm'].dt.year
df['month'] = df['call119_train.tm'].dt.month
df['day'] = df['call119_train.tm'].dt.day
# df['day_of_week'] = df['call119_train.tm'].dt.dayofweek  # 월요일=0, 일요일=6
# df['day_of_year'] = df['call119_train.tm'].dt.dayofyear
# df['week_of_year'] = df['call119_train.tm'].dt.isocalendar().week.astype(int)
# df['quarter'] = df['call119_train.tm'].dt.quarter

# 주기적 특성을 위한 Sine/Cosine 변환 (모델이 순환성을 학습하는 데 도움을 줌)
# df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
# df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
# df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
# df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)
# print("날짜 특성 생성 완료.")

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# --- 2. 특성(features) 목록 재정의 ---
# 원본 날짜('call119_train.tm')는 제외하고 새로 만든 날짜 특성을 포함시킵니다.
features = [col for col in df.columns if col not in [
    'Unnamed: 0', 'call119_train.call_count', 'call119_train.address_city',
    'call119_train.stn', 'call119_train.tm','day','month'# 원본 tm 컬럼 제외
]]
target = 'call119_train.call_count'

X_train = train_df[features].copy()
Y_train = train_df[target].copy()

X_test = test_df[features].copy()
Y_test = test_df[target].copy()




# # object 타입을 category로 변환 (LightGBM/XGBoost가 범주형으로 인식)
for col in X_train.select_dtypes(include='object').columns:
    # 훈련셋과 테스트셋의 모든 범주를 파악하여 누락되는 값이 없도록 처리
    all_categories = pd.concat([X_train[col], X_test[col]]).astype('category').cat.categories
    X_train[col] = pd.Categorical(X_train[col], categories=all_categories)
    X_test[col] = pd.Categorical(X_test[col], categories=all_categories)

# # # --- LightGBM 모델 학습 및 평가 ---
# # print("\nLGBM 모델을 학습합니다...")
lgb_model = lgb.LGBMRegressor(random_state=42)
# category 타입의 컬럼을 그대로 사용하여 학습
lgb_model.fit(X_train, Y_train)
Y_pred_lgb = lgb_model.predict(X_test)

rmse_lgb = np.sqrt(mean_squared_error(Y_test, Y_pred_lgb))
mae_lgb = mean_absolute_error(Y_test, Y_pred_lgb)


print(f"LGBMRegressor with Date Features RMSE: {rmse_lgb}")
print(f"LGBMRegressor MAE: {mae_lgb}")

# --- XGBoost 모델 학습 및 평가 ---
# XGBoost 1.5.0 이상 버전은 enable_categorical=True 옵션으로 category 타입을 직접 지원합니다.
# print("\nXGBoost 모델을 학습합니다...")
# xgb_model = xgb.XGBRegressor(random_state=42, enable_categorical=True, tree_method='hist')
# xgb_model.fit(X_train, Y_train)
# Y_pred_xgb = xgb_model.predict(X_test)
# rmse_xgb = np.sqrt(mean_squared_error(Y_test, Y_pred_xgb))
# print(f"XGBRegressor with Date Features RMSE: {rmse_xgb}")

날짜 특성을 생성합니다...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001240 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1849
[LightGBM] [Info] Number of data points in the train set: 32372, number of used features: 11
[LightGBM] [Info] Start training from score 1.950358
LGBMRegressor with Date Features RMSE: 1.1581444129694443
LGBMRegressor MAE: 0.7946775614941148


이거는 결측치를 재현님 알려주신 KNN inputter로 채워넣어서 했다. 일단 성능 자체는 결측치를 아무것도 안채워넣고 lightgbm이 알아서 처리하도록 하는것이 성능상 베스트로 나왔다. 각 모델마다 결측치를 처리하는 방법이 다르고, 어떻게 결측치를 처리하는냐에 따라 성능이 크게 좌우된다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/preprocessed_full_data_V3.csv'
df = pd.read_csv(file_path)


# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features].drop('call119_train.tm', axis=1)
Y_test = test_df[target]

# --- LightGBM (LGBMRegressor) 최적의 하이퍼파라미터 탐색 ---
print("\n--- LightGBM (LGBMRegressor) Hyperparameter Tuning ---")

# LGBMRegressor 모델 초기화
lgb_model = lgb.LGBMRegressor(random_state=42)

# 튜닝할 파라미터들의 조합을 정의합니다.
# 이 파라미터 그리드는 시작점으로 사용되며, 필요에 따라 더 넓거나 좁게 설정할 수 있습니다.
param_grid_lgb = {
    'n_estimators': [100, 200, 300], # 부스팅 단계 수 (트리 개수)
    'learning_rate': [0.01, 0.05, 0.1], # 각 부스팅 단계에서 학습률
    'num_leaves': [20, 31, 40], # 한 트리의 최대 리프 노드 수 (LightGBM 특유의 파라미터, 깊이와 관련)
    'max_depth': [-1, 10, 15], # 트리의 최대 깊이 (-1은 제한 없음)
    'reg_alpha': [0, 0.1, 0.5], # L1 정규화 항
    'reg_lambda': [0, 0.1, 0.5] # L2 정규화 항
}

# GridSearchCV 객체 생성
# cv: 교차 검증 폴드 수 (예: 5)
# scoring: 모델 성능 평가 지표 (회귀에서는 'neg_mean_squared_error'를 사용하면 MSE를 최소화)
# n_jobs: 사용할 CPU 코어 수 (-1은 모든 코어 사용)
# verbose: 학습 진행 상황 출력 레벨
grid_search_lgb = GridSearchCV(estimator=lgb_model,
                               param_grid=param_grid_lgb,
                               cv=5, # 5-겹 교차 검증
                               scoring='neg_mean_squared_error', # MSE를 최소화 (음수 값이므로 큰 값이 좋음)
                               n_jobs=-1, # 모든 코어 사용
                               verbose=2) # 진행 상황 출력

# 그리드 서치 수행
grid_search_lgb.fit(X_train, Y_train)

# 최적의 파라미터 조합 출력
print(f"\nBest LGBM parameters found: {grid_search_lgb.best_params_}")

# 최적의 모델로 예측 및 RMSE 계산
best_lgbm_model = grid_search_lgb.best_estimator_
Y_pred_lgbm_tuned = best_lgbm_model.predict(X_test)
rmse_lgbm_tuned = np.sqrt(mean_squared_error(Y_test, Y_pred_lgbm_tuned))
print(f"LGBMRegressor Tuned RMSE: {rmse_lgbm_tuned}")


--- LightGBM (LGBMRegressor) Hyperparameter Tuning ---
Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001026 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1922
[LightGBM] [Info] Number of data points in the train set: 32372, number of used features: 10
[LightGBM] [Info] Start training from score 1.950358

Best LGBM parameters found: {'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 200, 'num_leaves': 20, 'reg_alpha': 0, 'reg_lambda': 0}
LGBMRegressor Tuned RMSE: 1.2551551383312003


merged data로 각 카테고리별 분석후 성능을 더 뽑아낼려고 해도 성능이 오히려 더 떨어지는 역효과가 발생했습니다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/merged_data.csv'
df = pd.read_csv(file_path)



# --- 전처리 파이프라인 시작 ---

# 2. tm, address_city, stn 칼럼 제거
df = df.drop(columns=['tm', 'address_city', 'stn'])

# 3. -99 값을 np.nan으로 변환
df.replace(-99, np.nan, inplace=True)

# 4. address_gu, sub_address 라벨 인코딩
# KNNImputer는 숫자형 데이터에서만 작동하므로, 문자열 결측치를 먼저 채워야 합니다.
for col in ['address_gu', 'sub_address']:
    # NaN 값을 'Unknown' 같은 문자열로 채웁니다.
    df[col].fillna('Unknown', inplace=True)
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 5. 날짜 특성 공학
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df = df.drop(columns=['date'])

# 6. KNNImputer로 결측치 채우기
# 원본 데이터프레임의 컬럼 순서를 저장해둡니다.
original_columns = df.columns
imputer = KNNImputer(n_neighbors=5)
df_imputed = imputer.fit_transform(df)

# 다시 데이터프레임으로 변환
df = pd.DataFrame(df_imputed, columns=original_columns)

print("✅ 전처리가 완료되었습니다. 결측치가 모두 채워졌습니다.")
print(df.head())

# --- 모델링 및 예측 ---

# 7. 피처와 타겟 정의
cat_call_features = [
  'cat_call_교통사고', 'cat_call_부상', 'cat_call_업무운행', 'cat_call_산악사고',
  'cat_call_산불', 'cat_call_구급기타', 'cat_call_대민지원', 'cat_call_수난사고',
  'cat_call_상황출동', 'cat_call_자연재해', 'cat_call_기타', 'cat_call_지원출동(풍수해)'
]
predictor_columns = [col for col in df.columns if col not in cat_call_features and col != 'call_count']

X = df[predictor_columns]
y_final = df['call_count']

# 8. 데이터 분리
X_train, X_test, y_train_final, y_test_final = train_test_split(X, y_final, test_size=0.2, random_state=42)

# 9. 모델 학습 및 개별 예측
cat_call_predictions = {}
for category in cat_call_features:
    y_category = df[category]
    y_category_train = y_category.loc[X_train.index]

    model = LinearRegression()
    model.fit(X_train, y_category_train)

    predictions = model.predict(X_test)
    cat_call_predictions[category] = predictions

# 10. 최종 call_count 예측 (개별 예측의 합)
predicted_call_count = np.sum(list(cat_call_predictions.values()), axis=0)

# 11. 모델 평가 (RMSE, R-squared)
mse = mean_squared_error(y_test_final, predicted_call_count)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_final, predicted_call_count)

print("\n--- 최종 예측 평가 ---")
print(f"📈 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"📊 R-squared (R²): {r2:.4f}")

<ipython-input-16-988815491>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)
<ipython-input-16-988815491>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method(

✅ 전처리가 완료되었습니다. 결측치가 모두 채워졌습니다.
   address_gu  sub_address  ta_max  ta_min  ta_max_min  hm_min  hm_max  \
0         0.0         34.0    23.7    16.6         7.1   63.20   90.30   
1         0.0        121.0    24.3    15.4         8.9   64.20   96.40   
2         1.0         16.0    25.6    17.2         8.4   46.60   76.40   
3         1.0         17.0    25.3    17.1         8.2   49.42   85.54   
4         1.0         21.0    26.5    16.7         9.8   52.14   93.72   

   ws_max  ws_ins_max  rn_day  ...  cat_call_산악사고  cat_call_산불  cat_call_구급기타  \
0     5.8        10.6     0.0  ...            0.0          0.0            0.0   
1     4.8         8.6     0.0  ...            0.0          0.0            0.0   
2     4.8         8.3     0.0  ...            0.0          0.0            0.0   
3     7.1         9.7     0.0  ...            0.0          0.0            0.0   
4     5.7         8.1     0.0  ...            0.0          0.0            0.0   

   cat_call_대민지원  cat_call_수난사고  cat

merged data로 각 카테고리별 분석후 성능을 더 뽑아낼려고 해도 성능이 오히려 더 떨어지는 역효과가 발생했습니다.


In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/merged_data.csv'
df = pd.read_csv(file_path)



# --- 전처리 파이프라인 시작 ---

# 2. tm, address_city, stn 칼럼 제거
df = df.drop(columns=['tm', 'address_city', 'stn'])

# 3. -99 값을 np.nan으로 변환
df.replace(-99, np.nan, inplace=True)


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply LabelEncoder to 'call119_train.address_gu' and 'call119_train.sub_address'
df['address_gu'] = label_encoder.fit_transform(df['address_gu'])
df['sub_address'] = label_encoder.fit_transform(df['sub_address'])


features_for_imputation = [
    'ta_max',
    'ta_min',
    'ta_max_min',
    'hm_min',
    'hm_max',
    'ws_max',
    'ws_ins_max',
    'rn_day'
]

# 결측치 처리 전 스케일링
# StandardScaler 객체 생성
scaler = StandardScaler()

# KNNImputer를 적용할 특성들만 추출하여 스케일링
# 스케일링은 결측치가 없는 데이터에 먼저 fit되어야 하지만,
# KNNImputer가 NaN 값을 처리할 수 있으므로, 전체 데이터에 바로 적용합니다.
# 하지만, 일반적으로는 결측치를 먼저 처리한 후에 스케일링을 하는 것이 안전합니다.
# KNNImputer는 NaN 값을 무시하고 거리 계산을 수행하므로,
# 스케일링을 먼저 적용한 후 Impute 하는 것이 더 좋습니다 (NaN 값은 스케일링에 영향을 받지 않음).
df_scaled = pd.DataFrame(scaler.fit_transform(df[features_for_imputation]),
                         columns=features_for_imputation,
                         index=df.index)

# KNNImputer 객체 생성 (n_neighbors는 이웃의 수, 기본값은 5)
imputer = KNNImputer(n_neighbors=5, weights='uniform')

# 스케일링된 데이터에 KNNImputer 적용
df_imputed_scaled = pd.DataFrame(imputer.fit_transform(df_scaled),
                                 columns=features_for_imputation,
                                 index=df.index)

# Imputation이 완료된 데이터를 다시 원래 스케일로 되돌립니다 (역변환)
df_imputed_original_scale = pd.DataFrame(scaler.inverse_transform(df_imputed_scaled),
                                         columns=features_for_imputation,
                                         index=df.index)

# 원본 DataFrame에 결측치가 채워지고 원래 스케일로 복원된 특성들을 업데이트합니다.
for col in features_for_imputation:
    df[col] = df_imputed_original_scale[col]

# 결측치 처리 후 데이터 정보 확인
print("After KNN Imputation and Inverse Scaling, DataFrame Info:")
df.info()

# 확인을 위해 각 컬럼별 결측치 개수 출력
print("\nMissing values after KNN imputation per column:")
print(df.isnull().sum())



print("✅ 전처리가 완료되었습니다. 결측치가 모두 채워졌습니다.")
print(df.head())

# --- 모델링 및 예측 ---

# 7. 피처와 타겟 정의
cat_call_features = [
  'cat_call_교통사고', 'cat_call_부상', 'cat_call_업무운행', 'cat_call_산악사고',
  'cat_call_산불', 'cat_call_구급기타', 'cat_call_대민지원', 'cat_call_수난사고',
  'cat_call_상황출동', 'cat_call_자연재해', 'cat_call_기타', 'cat_call_지원출동(풍수해)'
]
predictor_columns = [col for col in df.columns if col not in cat_call_features and col != 'call_count']

X = df[predictor_columns]
y_final = df['call_count']

# 8. 데이터 분리
X_train, X_test, y_train_final, y_test_final = train_test_split(X, y_final, test_size=0.2, random_state=42)

# 9. 모델 학습 및 개별 예측
cat_call_predictions = {}
for category in cat_call_features:
    y_category = df[category]
    y_category_train = y_category.loc[X_train.index]

    model = LinearRegression()
    model.fit(X_train, y_category_train)

    predictions = model.predict(X_test)
    cat_call_predictions[category] = predictions

# 10. 최종 call_count 예측 (개별 예측의 합)
predicted_call_count = np.sum(list(cat_call_predictions.values()), axis=0)

# 11. 모델 평가 (RMSE, R-squared)
mse = mean_squared_error(y_test_final, predicted_call_count)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_final, predicted_call_count)

print("\n--- 최종 예측 평가 ---")
print(f"📈 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"📊 R-squared (R²): {r2:.4f}")

After KNN Imputation and Inverse Scaling, DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42924 entries, 0 to 42923
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   address_gu          42924 non-null  int64  
 1   sub_address         42924 non-null  int64  
 2   ta_max              42924 non-null  float64
 3   ta_min              42924 non-null  float64
 4   ta_max_min          42924 non-null  float64
 5   hm_min              42924 non-null  float64
 6   hm_max              42924 non-null  float64
 7   ws_max              42924 non-null  float64
 8   ws_ins_max          42924 non-null  float64
 9   rn_day              42924 non-null  float64
 10  cat_call_교통사고       42924 non-null  int64  
 11  cat_call_부상         42924 non-null  int64  
 12  cat_call_업무운행       42924 non-null  int64  
 13  cat_call_산악사고       42924 non-null  int64  
 14  cat_call_산불         42924 non-null  int64  


재홍님이 제공해주신 merge_data 로 돌려봤는데 카테고리별(산불,교통사고,자연재해 등등)로 나눠서 예측해보고 나중에 합해서 총 신고 건수를 예측해보니 오히려 정확도가 좀더 떨어지게 나왔다.

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/merged_data.csv'
df = pd.read_csv(file_path)

# --- 전처리 파이프라인 시작 ---

# 2. 칼럼 제거
df = df.drop(columns=['tm', 'address_city', 'stn'])

# 3. -99 값을 np.nan으로 변환
df.replace(-99, np.nan, inplace=True)

# 4. 라벨 인코딩 (결측치 처리 포함)
for col in ['address_gu', 'sub_address']:
    # LabelEncoder는 NaN 값을 처리할 수 없으므로, 먼저 문자열로 채웁니다.
    df[col] = df[col].astype(str).fillna('Unknown')
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# 5. KNN Imputation을 위한 스케일링 및 결측치 처리
features_for_imputation = [
    'ta_max', 'ta_min', 'ta_max_min', 'hm_min',
    'hm_max', 'ws_max', 'ws_ins_max', 'rn_day'
]

# StandardScaler와 KNNImputer 객체 생성
scaler = StandardScaler()
imputer = KNNImputer(n_neighbors=5)

# 스케일링 -> 결측치 채우기 -> 역스케일링
df_to_impute = df[features_for_imputation]
df_scaled = scaler.fit_transform(df_to_impute)
df_imputed_scaled = imputer.fit_transform(df_scaled)
df_imputed_original_scale = scaler.inverse_transform(df_imputed_scaled)

# 원본 데이터프레임에 결측치가 채워진 값으로 업데이트
df[features_for_imputation] = df_imputed_original_scale

# # 6. 날짜 특성 생성
# if 'date' in df.columns:
#     df['date'] = pd.to_datetime(df['date'])
#     df['year'] = df['date'].dt.year
#     df['month'] = df['date'].dt.month
#     df['day'] = df['date'].dt.day
#     df = df.drop(columns=['date'])


print("✅ Preprocessing is complete. All missing values have been imputed.")

# --- 모델링 및 예측 ---

# 7. 피처와 타겟 정의
cat_call_features = [
  'cat_call_교통사고', 'cat_call_부상', 'cat_call_업무운행', 'cat_call_산악사고',
  'cat_call_산불', 'cat_call_구급기타', 'cat_call_대민지원', 'cat_call_수난사고',
  'cat_call_상황출동', 'cat_call_자연재해', 'cat_call_기타', 'cat_call_지원출동(풍수해)'
]
predictor_columns = [col for col in df.columns if col not in cat_call_features and col != 'call_count']

X = df[predictor_columns]
y_final = df['call_count']

# 8. 데이터 분리
X_train, X_test, y_train_final, y_test_final = train_test_split(X, y_final, test_size=0.2, random_state=42)

# 9. LightGBM 모델 학습 및 개별 예측
cat_call_predictions = {}
for category in cat_call_features:
    y_category = df[category]
    y_category_train = y_category.loc[X_train.index]

    # 모델을 LinearRegression에서 LGBMRegressor로 변경
    model = lgb.LGBMRegressor(random_state=42)
    model.fit(X_train, y_category_train)

    predictions = model.predict(X_test)
    cat_call_predictions[category] = predictions

# 10. 최종 call_count 예측 (개별 예측의 합)
predicted_call_count = np.sum(list(cat_call_predictions.values()), axis=0)

# 11. 모델 평가 (RMSE, R-squared)
mse = mean_squared_error(y_test_final, predicted_call_count)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_final, predicted_call_count)

print("\n--- Final Prediction Evaluation (LightGBM) ---")
print(f"📈 Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"📊 R-squared (R²): {r2:.4f}")

✅ Preprocessing is complete. All missing values have been imputed.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001180 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1913
[LightGBM] [Info] Number of data points in the train set: 34339, number of used features: 10
[LightGBM] [Info] Start training from score 0.523079
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004906 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1913
[LightGBM] [Info] Number of data points in the train set: 34339, number of used features: 10
[LightGBM] [Info] Start training from score 0.732433
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003764 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total

여기부분은 그냥 넘어가라.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # 재사용
from autogluon.tabular import TabularPredictor

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/call119_train1.csv'
df = pd.read_csv(file_path)
df.replace(-99, np.nan, inplace=True)

df = df.drop(columns=[ 'call119_train.address_city', 'call119_train.stn'])


# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
features = [col for col in df.columns if col not in ['Unnamed: 0', 'year', 'call119_train.call_count']]
target = 'call119_train.call_count'

X_train = train_df[features].drop('call119_train.tm', axis=1)
Y_train = train_df[target]

X_test = test_df[features].drop('call119_train.tm', axis=1)
Y_test = test_df[target]


predictor = TabularPredictor(label=target, eval_metric='root_mean_squared_error', path='AutogluonModels/call_count_prediction')

# Train the predictor
# AutoGluon will automatically perform feature engineering, model selection, and hyperparameter tuning.
predictor.fit(train_data=train_data, presets='best_quality')

# Make predictions on the test set
# For evaluation, we pass the test_data directly. AutoGluon will extract features and make predictions.
predictions = predictor.predict(test_data_features_only)

# Evaluate the model on the test set
# AutoGluon can also evaluate the performance on a provided dataset
leaderboard = predictor.leaderboard(test_data, silent=True)
print(leaderboard)

# You can also get the overall test score directly
test_performance = predictor.evaluate(test_data, silent=True)
print(f"Test performance (RMSE): {test_performance['root_mean_squared_error']}")

print("\nAutoGluon has completed training and evaluation.")
print("The 'predictions' variable contains the predicted call counts for the 2023 data.")





NameError: name 'train_data' is not defined

이거 깔아야 마지막 셀이 돌아간다.

In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort 

AutoGluon 이라고 거의 모든 머신러닝 모델이랑 딥러닝 모델중에서 제일 성능이 괜찮은 모델로 자동 조합해주는 라이브러리이다. Firecast 팀즈단체방에 올려준 call119_train1.csv 만으로 돌려봤다. AutoGluon은 결측치를 알아서 처리해주기 때문에 결측치인 -99를 np.nan 으로 바꿔놓기만 하면 된다. 그리고 문자열 칼럼들을 정수로 인코딩해줘야 되는 작업들을 굳이 안해도 autogluon이 알아서 해준다는 장점이 있다.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error # For manual RMSE calculation
from autogluon.tabular import TabularPredictor

# Load the preprocessed dataset
file_path = '/content/drive/MyDrive/AIVLE/contest_for_firecast/call119_train1.csv'
df = pd.read_csv(file_path)
df.replace(-99, np.nan, inplace=True)

# Drop unnecessary columns
df = df.drop(columns=['call119_train.address_city', 'call119_train.stn'])



# Extract year from 'call119_train.tm'
df['year'] = df['call119_train.tm'].astype(str).str[:4].astype(int)

# Separate data into train and test based on year
train_df = df[df['year'].isin([2020, 2021, 2022])].copy()
test_df = df[df['year'] == 2023].copy()

# Define features (X) and target (Y)
# The 'Unnamed: 0' column is typically an index and should not be a feature.
# The 'year' column was used for splitting and also shouldn't be a feature for the model.
features = [col for col in df.columns if col not in ['Unnamed: 0', 'call119_train.call_count']]
target = 'call119_train.call_count'

# Prepare the training data for AutoGluon
# AutoGluon's `fit` method expects a single DataFrame containing both features and the target.
# We'll include the target column 'call119_train.call_count' in 'train_data_autogluon'.
train_data_autogluon = train_df[features + [target]].drop('call119_train.tm', axis=1)



# Prepare the test data for AutoGluon prediction and evaluation
# For prediction, you pass only the features.
# For evaluation (using AutoGluon's .evaluate()), you pass features + target.
test_data_features = test_df[features].drop('call119_train.tm', axis=1)
Y_test_actual = test_df[target] # Store the actual target values for RMSE calculation

# --- AutoGluon Model Training and Evaluation ---

# Initialize the AutoGluon TabularPredictor
# 'label' specifies the target column.
# 'eval_metric' tells AutoGluon what metric to optimize for and report.
# 'path' defines where AutoGluon will save its models and logs.
predictor = TabularPredictor(
    label=target,
    eval_metric='root_mean_squared_error', # Set evaluation metric to RMSE
    problem_type='regression',
    path='AutogluonModels/firecast_prediction' # Directory to save model artifacts
)

# Train the predictor
# We pass the combined 'train_data_autogluon' which includes both features and the target.
# 'presets='best_quality'' will train a variety of models for optimal performance.
print("\nStarting AutoGluon training...")
predictor.fit(
    train_data=train_data_autogluon,
    presets='best_quality',
    time_limit=3600 # Optional: set a time limit for training in seconds (e.g., 1 hour)
)

# Make predictions on the test set features (X_test equivalent)
print("\nMaking predictions on the 2023 test data...")
predictions = predictor.predict(test_data_features)

# Calculate RMSE manually using the predictions and the actual Y_test values
rmse_manual = np.sqrt(mean_squared_error(Y_test_actual, predictions))
print(f"\nCalculated RMSE on 2023 test data: {rmse_manual:.4f}")

# Optional: Use AutoGluon's built-in evaluate method for a more comprehensive report
# For this, you need to provide a DataFrame with both features and the actual target.
# test_data_for_autogluon_eval = pd.concat([test_data_features, Y_test_actual], axis=1)
# evaluation_results = predictor.evaluate(test_data_for_autogluon_eval, silent=True)
# print("\nAutoGluon's detailed evaluation on 2023 test data:")
# print(f"Overall Test Performance (RMSE from AutoGluon): {evaluation_results['root_mean_squared_error']:.4f}")

# print("\nAutoGluon process complete!")
# print(f"First 5 predictions:\n{predictions.head()}")

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       10.83 GB / 12.67 GB (85.5%)
Disk Space Avail:   68.03 GB / 107.72 GB (63.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack


Starting AutoGluon training...


	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-06-16 15:54:17,935	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
		Context path: "/content/AutogluonModels/firecast_prediction/ds_sub_fit/sub_fit_ho"
(_dystack pid=23375) Running DyStack sub-fit ...
(_dystack pid=23375) Beginning AutoGluon training ... Time limit = 894s
(_dystack pid=23375) AutoGluon will save models to "/content/AutogluonModels/firecast_prediction/ds_sub_fit/sub_fit_ho"
(_dystack pid=23375) Train Data Rows:    28775
(_dystack pid=23375) Train Data Columns: 11
(_dystack pid=23375) Label Column:       call119_train.call_count
(_dystack pid=23375) Problem Type:       regression
(_dystack pid=23375) Preprocessing data ...
(_dystack pid=23375) Using Feature Generators to preprocess the data ...
(_dystack pid=23375) Fitting 

(_ray_fit pid=23597) [1000]	valid_set's rmse: 1.7651
(_ray_fit pid=24052) [1000]	valid_set's rmse: 2.20147
(_ray_fit pid=24052) [2000]	valid_set's rmse: 2.17335


(_dystack pid=23375) 	-1.6508	 = Validation score   (-root_mean_squared_error)
(_dystack pid=23375) 	57.78s	 = Training   runtime
(_dystack pid=23375) 	3.48s	 = Validation runtime
(_dystack pid=23375) Fitting model: LightGBM_BAG_L1 ... Training model for up to 526.85s of the 824.89s of remaining time.
(_dystack pid=23375) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.19%)


...

(_ray_fit pid=28399) [1000]	valid_set's rmse: 1.29774
(_ray_fit pid=28398) [2000]	valid_set's rmse: 1.76656 [repeated 2x across cluster]
(_ray_fit pid=28398) [3000]	valid_set's rmse: 1.76292
(_ray_fit pid=28398) [4000]	valid_set's rmse: 1.76083
(_ray_fit pid=28398) [5000]	valid_set's rmse: 1.7603


(_ray_fit pid=28398) 	Ran out of time, early stopping on iteration 5945. Best iteration is:
(_ray_fit pid=28398) 	[5706]	valid_set's rmse: 1.75855
(_ray_fit pid=28124) 	Ran out of time, stopping training early. (Stopping on epoch 7)
(_dystack pid=23375) 	-1.6093	 = Validation score   (-root_mean_squared_error)
(_dystack pid=23375) 	77.18s	 = Training   runtime
(_dystack pid=23375) 	7.06s	 = Validation runtime
(_dystack pid=23375) Fitting model: LightGBM_BAG_L2 ... Training model for up to 185.54s of the 185.50s of remaining time.
(_dystack pid=23375) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=0.27%)


(_ray_fit pid=29151) [1000]	valid_set's rmse: 1.36773
(_ray_fit pid=29151) [2000]	valid_set's rmse: 1.3544
(_ray_fit pid=29151) [3000]	valid_set's rmse: 1.35119
(_ray_fit pid=29151) [4000]	valid_set's rmse: 1.34955


(_ray_fit pid=29151) 	Ran out of time, early stopping on iteration 4039. Best iteration is:
(_ray_fit pid=29151) 	[3978]	valid_set's rmse: 1.34947
(_dystack pid=23375) 	-1.5878	 = Validation score   (-root_mean_squared_error)
(_dystack pid=23375) 	70.1s	 = Training   runtime
(_dystack pid=23375) 	2.95s	 = Validation runtime
(_dystack pid=23375) Fitting model: RandomForestMSE_BAG_L2 ... Training model for up to 111.71s of the 111.66s of remaining time.
(_dystack pid=23375) /usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
(_dystack pid=23375)   warnings.warn(
(_dystack pid=23375) 	-1.5254	 = Validation score   (-root_mean_squared_error)
(_dystack pid=23375) 	204.18s	 = Training   runtime
(_dystack pid=23375) 	1.51s	 = Validation runtime
(_dystack pid=2337


Making predictions on the 2023 test data...

Calculated RMSE on 2023 test data: 1.1252
